
# 使用Python调用OpenAI API构建聊天机器人

在之前的实验中，我们学习了如何在本地运行LLM。现在，我们将学习如何通过官方API与OpenAI强大的语言模型进行交互，特别是使用其为对话设计的 `chat completions` 端点。

## 1. 准备工作

在运行本笔记之前，你需要先设置好对OpenAI API的访问权限。

### 1.1 获取OpenAI API密钥

要使用OpenAI API，你必须拥有一个API密钥：

1. 在 [OpenAI官网](https://openai.com/) 创建一个账户。
2. 导航至 [API密钥页面](https://platform.openai.com/api-keys)。
3. 创建一个新的密钥。
4. **请务必妥善保管好这个密钥**——它就像你的密码一样重要！

### 1.2 配置你的环境

首先，安装OpenAI的Python官方库：

In [ ]:
%%bash
pip install openai python-dotenv

然后，设置你的API密钥。为了安全起见，最佳实践是使用环境变量来管理密钥。我们推荐创建一个名为 `.env` 的文件来存放它。

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# 从.env文件加载环境变量
# 请确保你的.env文件中有这样一行：OPENAI_API_KEY=sk-xxxxxxxxxx 
load_dotenv()

# 初始化OpenAI客户端。它会自动从环境变量中读取API密钥。
client = OpenAI()

## 2. 使用Chat Completions API

OpenAI提供的 `Chat Completions API` 专为对话式交互而设计。调用它时，我们需要提供一个消息列表，其中通常包含**系统消息**（设定模型的行为方式）和**用户消息**（用户的输入）。

In [ ]:
# 我们使用 client.chat.completions.create 方法，它接收一个消息列表（messages）和模型名称（model）。
# 消息列表是一个由字典组成的数组，每个字典都包含 'role' 和 'content' 两个键。
# 第一个消息通常是 'system' 角色的系统消息，用于设定助手的行为。
# 后续消息是 'user' 角色的用户输入。
response = client.chat.completions.create(
    model="gpt-4o", # 你也可以选择 gpt-3.5-turbo 等其他模型
    messages=[
        {"role": "system", "content": "你是一个乐于助人的助手。"},
        {"role": "user", "content": "写一个关于独角兽的单句睡前故事。"}
    ]
)

# 响应对象 `response` 包含一个 `choices` 列表，我们通常取第一个选择（choices[0]）。
# 然后从该选择的 `message` 属性中获取我们想要的 `content`。
print(response.choices[0].message.content)

## 3. 理解响应结构

API返回的是一个结构化的响应对象，其中包含了许多有用的元数据。让我们打印出来看看：

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "你是一个乐于助人的助手。"},
        {"role": "user", "content": "你好，你今天过得怎么样？"}
    ]
)

# 打印完整的响应对象结构
print("完整的响应对象:")
print(response)

# 访问特定的部分
print("
仅消息内容:")
print(response.choices[0].message.content)

print("
使用的模型:")
print(response.model)

print("
使用量统计:")
print(f"输入令牌数 (Prompt tokens): {response.usage.prompt_tokens}")
print(f"输出令牌数 (Completion tokens): {response.usage.completion_tokens}")
print(f"总令牌数 (Total tokens): {response.usage.total_tokens}")

从响应中，我们可以看到许多有用的数据。最重要的是，我们能清楚地了解这次API调用消耗了多少**令牌（tokens）**。模型的定价通常基于输入和输出令牌的总和，所以在本例中，我们为54个令牌付费。我们将在本实验的最后更详细地讨论定价问题。

## 4. 创建一个简单的聊天界面（无记忆）

让我们先构建一个没有记忆功能的聊天机器人。

在Jupyter Notebook环境中，我们无法创建一个持续等待输入的聊天循环（即 `input()` 循环）。因此，我们将创建一个简单的函数，并在不同的单元格中调用它，以模拟一次性的、无记忆的聊天交互。

### 单元格 1: 定义函数

In [ ]:
# 这个函数接收一个用户消息作为输入，并从OpenAI API返回一个响应。
def get_response_no_memory(user_message):
    """从OpenAI获取响应（不包含对话历史）"""
    # 这是一个文档字符串（docstring），用于描述函数的功能和参数，对于代码的可读性和文档生成非常有用。

    # 我们用用户消息调用模型
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "你是一个乐于助人的助手。"},
            {"role": "user", "content": user_message}
        ]
    )
    
    return response.choices[0].message.content

### 单元格 2: 第一次交互

In [ ]:
# 第一个问题（你可以输入任何内容）
question = "什么是人工智能？"
answer = get_response_no_memory(question)

print(f"你: {question}")
print(f"助手: {answer}")

### 单元格 3: 后续问题（无记忆）

In [ ]:
# 第二个问题（AI不会记得上一次的交互）
question = "你能就此详细阐述一下吗？"
answer = get_response_no_memory(question)

print(f"你: {question}")
print(f"助手: {answer}")

你会注意到，当你问“你能就此详细阐述一下吗？”，助手不知道“此”指的是什么，因为它对上一次的交流**毫无记忆**。

## 5. 创建一个有记忆的聊天界面

现在，让我们构建一个能够维护对话历史的版本。

### 单元格 1: 设置对话记忆

In [ ]:
# 用系统消息初始化对话列表。我们将随着对话的进行，向这个列表中添加更多的消息。
conversation_memory = [
    {"role": "system", "content": "你是一个乐于助人的助手。"}
]

# 定义一个函数，用于将用户消息添加到历史记录中，并从OpenAI获取响应
def chat_with_memory(user_message):
    """在维护对话历史的同时与AI聊天"""
    
    # 1. 将用户的消息添加到历史记录中
    conversation_memory.append({"role": "user", "content": user_message})
    
    # 2. 使用完整的对话历史调用OpenAI API
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=conversation_memory
    )
    
    # 3. 提取助手的响应内容
    assistant_response = response.choices[0].message.content
    
    # 4. 将助手的响应也添加到历史记录中，为下一次交互做准备
    conversation_memory.append({"role": "assistant", "content": assistant_response})
    
    # 5. 返回响应内容和本次交互消耗的令牌数
    return assistant_response, response.usage.total_tokens

# 定义一个函数来显示对话历史。我们遍历历史列表并打印每条消息，跳过系统消息以保持输出整洁。
def show_conversation():
    """显示当前的对话内容"""
    for message in conversation_memory:
        if message["role"] == "system":
            continue  # 跳过系统消息
        # 将角色首字母大写以美化输出，例如 'user' -> 'User'
        print(f"{message['role'].capitalize()}: {message['content']}\n")

### 单元格 2: 第一次有记忆的交互

In [ ]:
# 第一个问题
question = "什么是人工智能？"
answer, tokens = chat_with_memory(question)

print(f"你: {question}")
print(f"助手: {answer}")
print(f"[消耗令牌数: {tokens}]")

### 单元格 3: 后续问题（有记忆）

In [ ]:
# 第二个问题（现在AI会记得上一次的交互了）
question = "你能就此详细阐述一下吗？"
answer, tokens = chat_with_memory(question)

print(f"你: {question}")
print(f"助手: {answer}")
print(f"[消耗令牌数: {tokens}]")

### 单元格 4: 查看完整对话

In [ ]:
# 查看至今为止的完整对话历史
print("完整对话历史:")
print("-" * 30)
show_conversation()

### 单元格 5: 重置对话（如果需要）

In [ ]:
# 如果你想重新开始一段对话，可以重置对话历史
conversation_memory = [
    {"role": "system", "content": "你是一个乐于助人的助手。"}
]
print("对话已重置！")

## 6. 流式响应 (Streaming Responses)

流式响应可以让你在模型生成答案的同时，实时地看到输出，就像打字机一样，而不是等待它一次性生成全部内容。

In [ ]:
import time

def stream_response(user_message):
    """从OpenAI流式获取响应，但不存储对话历史"""
    
    # 通过设置 `stream=True`，API会以数据块（chunks）的形式返回响应。
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "你是一个乐于助人的助手。"},
            {"role": "user", "content": user_message}
        ],
        stream=True
    )
    
    print(f"你: {user_message}")
    print("助手: ", end="", flush=True)  # `end=""` 表示不换行，`flush=True` 确保立即打印输出。
    
    # 处理数据流
    full_response = "" # 初始化一个空字符串，用于拼接完整的响应。
    for chunk in stream: # 遍历数据流中的每一个数据块。
        # 检查数据块中是否有内容。
        if chunk.choices[0].delta.content is not None:
            content_chunk = chunk.choices[0].delta.content # 获取这部分响应内容。
            full_response += content_chunk # 将其拼接到完整响应中。
            print(content_chunk, end="", flush=True) # 实时打印这部分内容。
            time.sleep(0.01)  # 添加微小延迟，使输出更易读。
    
    print("\n")  # 响应结束后换行。
    return full_response

测试流式函数：

In [ ]:
# 尝试流式函数
stream_response("写一首关于编程的短诗。")

## 7. 带记忆的流式响应

让我们将流式响应与对话记忆结合起来：

In [ ]:
# 初始化带记忆的流式对话历史
streaming_conversation = [
    {"role": "system", "content": "你是一个乐于助人的助手。"}
]

def stream_chat_with_memory(user_message):
    """带记忆的流式聊天"""
    
    # 将用户消息添加到历史记录
    streaming_conversation.append({"role": "user", "content": user_message})
    
    # 获取流式响应
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=streaming_conversation,
        stream=True
    )
    
    print(f"你: {user_message}")
    print("助手: ", end="", flush=True)
    
    # 处理数据流
    assistant_response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content_chunk = chunk.choices[0].delta.content
            assistant_response += content_chunk
            print(content_chunk, end="", flush=True)
            time.sleep(0.01)
    
    print("\n")  # 响应结束后换行
    
    # 将助手的完整响应添加到历史记录
    streaming_conversation.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response

测试带记忆的流式聊天：

In [ ]:
# 第一个带记忆的流式问题
stream_chat_with_memory("机器人三定律是什么？")

In [ ]:
# 后续的带记忆的流式问题
stream_chat_with_memory("是谁创造了这些定律？")

## 8. 理解不同的消息角色

OpenAI Chat API在消息中主要使用三种角色：

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        # 系统 (system) 消息 - 设定行为和上下文
        {"role": "system", "content": "你是一个只会用海盗黑话说话的海盗。"},
        
        # 用户 (user) 消息 - 用户说的话
        {"role": "user", "content": "你好，今天过得怎么样？"},
        
        # 助手 (assistant) 消息 - 助手之前的回复，用于提供对话历史
        {"role": "assistant", "content": "啊哈！我今天感觉棒极了，我的好伙计！"},
        
        # 另一个用户消息
        {"role": "user", "content": "跟我说说天气怎么样。"}
    ]
)

print(response.choices[0].message.content)

## 9. 理解上下文窗口

不同的OpenAI模型有不同的上下文窗口限制：

- **GPT-3.5-Turbo**: 4,096 或 16,384 令牌（取决于具体版本）
- **GPT-4**: 8,192 或 32,768 令牌（取决于具体版本）
- **GPT-4-Turbo / GPT-4o**: 高达 128,000 令牌

与本地模型不同，OpenAI会为你管理令牌：
1. 如果你发送的对话历史超出了模型的上下文窗口限制，API将返回错误。
2. 你的费用是根据你使用的令牌数量计算的。
3. API在每次请求的响应中都会提供令牌使用统计。

让我们看看令牌在实际中的作用：

In [ ]:
# 创建一个更长的对话
long_messages = [
    {"role": "system", "content": "你是一个乐于助人的助手。"}
]

# 向历史记录中添加一些消息
for i in range(5):
    long_messages.append({"role": "user", "content": f"这是测试消息 {i+1}。告诉我一些关于太空的有趣事实。"})
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=long_messages
    )
    assistant_msg = response.choices[0].message.content
    long_messages.append({"role": "assistant", "content": assistant_msg})
    print(f"第 {i+1} 轮交互 - 总令牌数: {response.usage.total_tokens}")

## 10. 管理成本和令牌

使用OpenAI API时，你需要时刻关注成本：

1. **令牌计量**: 每次请求和响应都会消耗令牌，而你需要为这些令牌付费。
2. **模型选择**: 更强大的模型（如GPT-4o）通常比基础模型（如GPT-3.5-Turbo）每令牌的成本更高。
3. **上下文窗口**: 更长的对话会发送更多的令牌，因此成本也更高。

管理成本的一些技巧：

In [ ]:
# 1. 对复杂度较低的任务使用更便宜的模型
response_cheap = client.chat.completions.create(
    model="gpt-3.5-turbo",  # 比 GPT-4o 便宜
    messages=[{"role": "user", "content": "总结一下锻炼的好处。"}]
)
print(f"GPT-3.5-Turbo 成本: {response_cheap.usage.total_tokens} 令牌")

# 2. 控制最大令牌数以限制响应长度和成本
response_limited = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "给我讲讲量子物理学。"}],
    max_tokens=100  # 限制响应长度最多为100个令牌
)
print(f"受限响应成本: {response_limited.usage.total_tokens} 令牌")

# 3. 使用温度（temperature）控制随机性。较高的值使输出更具创造性，较低的值使其更具确定性。
response_creative = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "写一个有创意的故事。"}],
    temperature=0.8  # 较高的温度以获得更有创意的输出
)
print(f"创意响应成本: {response_creative.usage.total_tokens} 令牌")

## 11. 处理长对话的对话历史

对于非常长的对话，你需要采取策略来管理上下文窗口，以避免超出限制并控制成本。

In [ ]:
# 策略1: 滑动窗口 - 只保留最近的N条消息
def trim_conversation(messages, max_messages=10):
    # 总是保留第一条系统消息
    if len(messages) > max_messages + 1:
        system_message = messages[0]
        recent_messages = messages[-(max_messages):] # 取最后N条消息
        return [system_message] + recent_messages
    return messages

# 策略2: 对话摘要 - 定期用AI总结对话，并用摘要替换旧消息
def summarize_conversation(messages):
    # 将对话历史拼接成一个字符串
    conversation_text = "\n".join([f"{m['role']}: {m['content']}" for m in messages if m['role'] != 'system'])
    
    # 创建一个摘要请求
    summary_request = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "请简明扼要地总结以下对话。"},
            {"role": "user", "content": conversation_text}
        ]
    )
    summary = summary_request.choices[0].message.content
    
    # 用摘要替换原始对话
    return [
        messages[0],  # 保留原始的系统消息
        {"role": "system", "content": f"先前对话的摘要: {summary}"}
    ]

# 何时使用：
# if len(conversation_memory) > 20: # 例如，当对话超过20条消息时
#     conversation_memory = summarize_conversation(conversation_memory)

## 12. 本地LLM vs. OpenAI API 对比

| 特性 | 本地LLM (Ollama) | OpenAI API |
|---|---|---|
| **设置** | 下载模型到本地，配置复杂 | 仅需API密钥，简单快捷 |
| **成本** | 硬件成本，无按次付费 | 按令牌使用量付费 |
| **隐私** | 数据保留在本地设备上 | 数据发送到OpenAI服务器 |
| **性能** | 受限于本地硬件 | 可使用最先进的模型 |
| **可靠性** | 取决于你的系统稳定性 | 由OpenAI管理的高可用服务 |
| **上下文窗口** | 通常较小 | 高达128K甚至更多令牌 |
| **内存管理** | 需要手动实现 | API层面部分处理，但仍需策略 |

## 13. 进一步学习的资源

- [OpenAI API 官方文档](https://platform.openai.com/docs/api-reference)
- [OpenAI Cookbook (代码示例库)](https://github.com/openai/openai-cookbook)
- [OpenAI Python 库](https://github.com/openai/openai-python)
- [OpenAI 令牌计算器](https://platform.openai.com/tokenizer)